# Advanced Macro II - Investment


In [ ]:
# Import necessary libraries
# Plotting library, including specific imports for animations
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.animation as animation
# Numerical computing library
import numpy as np
# Special option to display animations within a Colab notebok
rc('animation', html='jshtml')
# Optimzier
from scipy import optimize

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set the exogenous variables
r = 0.04      # Interest rate
alpha = 0.4   # Capital share of output
A = 1         # Total factor productivity
delta = 0.1  # Rate of depreciation

# Laws of motion of the economy
Recall that the equations governing the dynamics of the model are given by the optimality conditions.
These three equations build a *system of differential equations*. Solving such systems by hand is generally hard, however, with programming languages we can let computers do the heavy lifting for us.

In this notebook, we will take a look at a graphical solution for these equations, also known phase diagrams, as well as a numerical solution using Python. While there exist packages to solve differential equations such as [SciPy](https://www.scipy.org/), we will write our own implementation as this is more educational.


# Solving for the equilibrium values

In [ ]:
# Create a function for the q locus
def q_loc(x):
    return alpha * A * x**(alpha - 1) / (r + delta)

# Create a function for the K locus
def K_loc(x):
  return 1+ delta*x

# Compute K* using the above functions
def K_root(x):
  return q_loc(x)-K_loc(x)

K_star = optimize.fsolve(K_root, 1.0)

# Display the result
print(K_star)

In [ ]:
# q* is then determined by
q_star = K_loc(K_star)

# Display the result
print(q_star)


# Plotting the dynamics of $q$ and $K$
Now that we have computed the necessary information, we can set up the phase diagram


In [ ]:
# 1000 linearly spaced numbers between both points, this will be our x-axis
K = np.linspace(K_star*0.15, K_star*2, 1000)

# Set up a figure
fig, ax = plt.subplots(figsize = (12, 8))

# Add a blue line with x values given by K and y values given by q_loc(K)
ax.plot(K, q_loc(K), color = 'blue', label = r'$\dot{q}=0$')

# Add a green line with x values given by K and y values given by K_loc(x)
ax.plot(K, K_loc(K), color = 'green', label = r'$\dot{K}=0$')
ax.set_xlabel("K")                        # Add x-axis label
ax.set_ylabel("q")                        # Add y-axis label
ax.set_title("The phase diagram")   # Add plot title
ax.grid()                                 # Add a grid in the background
ax.legend()                               # Add legend (show labels top right)
plt.show()                                # Display the plot

We have not yet added the arrows which show the dynamics of the system. While we could add them manually, this is bothersome as we would need to write some code for each individual arrow. Instead, we'll make use of the tools `matplotlib` offers and create a so called *quiver plot*. Quiver plots are vector fields which can be used to show the dynamics of a system and are very common when dealing with differential equations.

Quiver plots allow us to plot a grid of arrows/vectors, where each vector represents both the direction (vector orientation) and intensity (vector length) of changes within the dynamical system.

In [ ]:
# In order to compute the direction and intensity of changes within the system,
# we need to set up a few functions allowing us to measure the underlying differential equations

# Create a function to compute the derivative of the production function
def f_prime(x):
  return alpha*A*x**(alpha-1)

# Create a function for the derivative of K(t) w.r.t time
def K_dot(K, q):
  return -1 + q - delta * K

# Create a function for the derivative of q(t) w.r.t. time
def q_dot(K, q):
  return (r+delta)*q - f_prime(K)

In [ ]:
# Set up the vector grid parameters
n_points = 10
# Create a grid (for the points on the plot where vectors are drawn)
x, y = np.meshgrid(
    np.linspace(min(K), max(K), n_points),
    np.linspace(q_star*0.1, K_loc(max(K))*2.2, n_points))
# Compute the x component of each vector
u = K_dot(x, y)
# Compute the y component of each vector
v = q_dot(x, y)
# Create a figure
fig, ax = plt.subplots(figsize = (14, 10))
# Add a blue line with x values given by k and y values given by q_loc
ax.plot(K, q_loc(K), color = 'blue', label = r'$\dot{q}=0$')
# Add a green line with x values given by k and y values given by K_loc
ax.plot(K, K_loc(K), color = 'green', label = r'$\dot{K}=0$')
# Add a single black point at (x, y) = (K*, q*), this is our equilibrium
ax.plot([K_star], [K_loc(K_star)], 'o', color = 'black')
# Annotate the point with the letter D
ax.annotate('D', xy = (K_star, K_loc(K_star)+0.1), size = 20)
# Add the vector field, showing how the dynamics of the system behave
ax.quiver(x, y, u, v, pivot = 'mid', color = 'gray', angles='xy',
           scale_units='xy', scale = 5, minlength = 0.02)
ax.set_xlabel("K")                        # Add x-axis label
ax.set_ylabel("q")                        # Add y-axis label
ax.set_title("The phase diagram with vector field")   # Add plot title
ax.grid()                                 # Add a grid in the background
ax.legend()                               # Add legend (show labels top right)

The above picture has the advantage of giving a better intuition about the full dynamics within the system. Indeed, it is much easier to see from this picture that some changes in the state of the economy will be much more drastic then others, depending on where the current state of the economy is.

We continue by building a function which, given an starting point $(K, q)$ on our plot, computes the evolution of the economy according to the law of motions defined above. The idea behind this function is a simple iteration through time, where at each time step we simply add the derivative $\dot{K}$ to the current $K$ and the derivative $\dot{q}$ to the current $q$.

In [ ]:
# Create a function which returns an array containing the trajectory of the
# economy starting a (K, q) for T time periods
def trajectory(K, q, T = 1000):
  # Initialize 2 arrays for the evolution of K and q
  K = [K]
  q = [q]
  # Iterate over the time periods and compute the evolution of k and c
  for t in range(T):
    # Compute the increments for K and q, saving them allows to reuse them
    # when checking the early stopping conditions
    Kd = K_dot(K[t], q[t])
    qd = q_dot(K[t], q[t])
    # Append the increments to the existing lists of values
    K.append(K[t] + Kd)
    q.append(q[t] + qd)
    # Early stopping condition
    if (Kd == qd == 0) or (K[-1] <= 0) or (q[-1] <= 0):
      break
  return np.asarray(K), np.asarray(q)

In [ ]:
# Finding a starting point which will end up in the equilibrium E is slightly more complicated
def reverse_trajectory(K_star, T = 150, eps = 0.01):
  # Initialize 2 arrays for the evolution of K and q but starting in [K_star, q_star]
  K = [K_star]
  q = [K_loc(K_star)]
  # Add a slight perturbation (±eps%) from the equilibrium to get a point with non-zero derivatives
  K.append(K[0] * (1 + eps))
  q.append(q[0] * (1 + eps))
  for t in range(1, T):
    # Compute the increments for k and c towards the equilibrium and subtract them
    # from the current position
    Kd = K_dot(K[t], q[t])
    qd = q_dot(K[t], q[t])
    K.append(K[t] - Kd)
    q.append(q[t] - qd)
  # Return the reversed arrays for K and q
  return np.asarray(K[::-1]), np.asarray(q[::-1])

In [ ]:
# Iterate over some 'randomly' chosen starting points and plot the evolution of the economy
# Notice we do not create a new figure, we simply add the trajectories to the existing
# figure we previously created
for Kq in [(2.0, 1.25, 'A'), (5.0, 0.4, 'B'), (4.0, 2.0, 'C')]:
  # Compute the x and y arrays using the trajectory function we defined above
  x, y = trajectory(Kq[0], Kq[1])
  ax.plot(x, y, '--', color = 'orange')         # Draw the paths as a dashed line
  ax.plot(Kq[0], Kq[1], 'o', color = 'orange')  # Add a dot at the starting point
  ax.annotate(Kq[2], xy = Kq[:2], size = 18)    # Add a letter at each starting point
# Add the trajectories which ends up in the equilibirum
x, y = reverse_trajectory(K_star, T = 150)
ax.plot(x, y, '--', color = 'purple')
ax.plot(x[115], y[115], 'o', color = 'purple')
ax.annotate('E', xy = (x[115], y[115]), size = 18)
x, y = reverse_trajectory(K_star, eps = -0.01, T = 160)
ax.plot(x, y, '--', color = 'purple')
ax.plot(x[125], y[125], 'o', color = 'purple')
ax.annotate('F', xy = (x[125], y[125]), size = 18)
# Restrain the x- and y-axis so that it cuts off the explosive paths
ax.set_ylim(bottom = 0, top = K_loc(max(K))*2.5)
ax.set_xlim(left = 0.4, right = max(K))
ax.set_title("The phase diagram with vector field and trajectories")   # Add plot title

fig                                             # Display the figure

## Animated plot
The above picture shows the direction for the path of the economy in different starting positions, but in a completely static manner.

We now add a time component using animations, i.e. we create an animation of how the economy develops over time starting in a given point.

In [ ]:
# Create a function to plot an animated evolution of the economy given a specific starting point
def animated_plot(K_start, q_start):
  # Create the plot
  fig = plt.figure(figsize = (12, 8))
  # Create a linear space of 1000 points between the two k values for which where c* = 0
  K = np.linspace(K_star*0.2, K_star*2, 1000)

  plt.plot(K, q_loc(K), color = 'blue', label = r'$\dot{q}=0$')
  plt.plot(K, K_loc(K), color = 'green', label =r'$\dot{K}=0$')
  plt.xlabel("K")                       # Add x-axis label
  plt.ylabel("q")                       # Add y-axis label
  plt.title("The dynamics of q and K")  # Add plot title
  plt.grid()                            # Add a grid
  # Compute the evolution of K and q using our previously defined function
  K, q = trajectory(K_start, q_start, 1000)
  # Plot a dashed orange line (notice how it is initialized with empty arrays)
  evolution_line,  = plt.plot([], [], '--', label = 'Economy trajectory',
                              color = 'orange')
  # Define how the evolution_line (the orange dashed line) evolves as a function of time
  def animate(t):
    # Instead of taking steps with linearly increasing size, we use a bit of a
    # trick to take smaller steps in the start and larger steps as time goes by
    step = int((t/50)**3 * len(K))
    # Update the evolution_line data
    evolution_line.set_data(K[:step], q[:step])
    return evolution_line,
  # Add legend and show the plot
  plt.legend()
  return animation.FuncAnimation(fig, animate, frames = 50, interval = 20, blit = True)

In [ ]:
# Create an animation of the economy's evolution starting at a point
K_start = 5
q_start = 1.6
# Change the parameters of the function animated_plot() to see how different starting points behave
anim = animated_plot(K_start, q_start)
plt.close()
anim

# Illustrating a Permanent Increase in the Output Elasticity of Capital

In [ ]:
# Compute the initial steady-state equilibrium values
K_star_old = optimize.fsolve(K_root, 1.0)
q_star_old = K_loc(K_star)

# Set alpha to its new value
alpha_new = 0.5




In [ ]:
#@title Updating the previous functions

# Get the new steady-state values of K and q
def K_loc_new(x):
  return 1+ delta*x

def q_loc_new(x):
  return alpha_new * A * x**(alpha_new - 1) / (r + delta)

def K_root_new(x):
  return q_loc_new(x)-K_loc_new(x)



# Update functions for calculating the trajectories
def f_prime_new(x):
  return alpha_new*A*x**(alpha_new-1)

def q_dot_new(K, q):
  return (r+delta)*q - f_prime_new(K)

def K_dot_new(K, q):
  return -1 + q - delta * K




In [ ]:
# Calculate the new equilibrium
K_star_new = optimize.fsolve(K_root_new, 5)
q_star_new = K_loc_new(K_star_new)

In [ ]:
#@title Some more function updating and preparing the animated plot

# Finding a starting point which will end up in the equilibrium
def reverse_trajectory_new(K_star_new, T = 100, eps = 0.01):
  # Initialize 2 arrays for the evolution of K and q but starting in [K_star, q_star]
  K = [K_star_new]
  q = [K_loc_new(K_star_new)]
  # Add a slight perturbation (±eps%) from the equilibrium to get a point with non-zero derivatives
  K.append(K[0] * (1 + eps))
  q.append(q[0] * (1 + eps))
  for t in range(1, T):
    # Compute the increments for k and c towards the equilibrium and subtract them
    # from the current position
    Kd = K_dot_new(K[t], q[t])
    qd = q_dot_new(K[t], q[t])
    K.append(K[t] - Kd)
    q.append(q[t] - qd)
  # Return the reversed arrays for K and q
  return np.asarray(K[::-1]), np.asarray(q[::-1])



def lin_intpl_q(K, q, K_mid):
    for i in range(len(K) - 1):
        if K[i] <= K_mid <= K[i + 1]:
            lower = K[i]
            upper = K[i + 1]

            q_lower = q[i]
            q_upper = q[i + 1]

            interpolated_q = q_lower + (K_mid - lower) * (q_upper - q_lower) / (upper - lower)
            return i, interpolated_q
    return None, None



def animated_plot_new(K_trans, q_trans):
  # Create the plot
  fig = plt.figure(figsize = (12, 8))
  # Create a linear space of 1000 points between the two k values for which where c* = 0
  K = np.linspace(K_star_old*0.2, K_star_new*2, 1000)

  plt.plot(K, K_loc(K), color = 'green', label =r'$\dot{K}=0$')
  plt.plot(K, q_loc(K), color = 'black', label = r'$\dot{q}=0$')
  plt.plot(K_star_old, q_star_old, 'o', color = 'black')
  plt.annotate('A', xy = (K_star_old, K_loc(K_star_old)-0.2), size = 20)
  plt.plot(K, q_loc_new(K), color = 'blue', label = r'$\dot{q}_{new}=0$')
  plt.plot(K_star_new, q_star_new, 'o', color = 'blue')
  plt.annotate('B', xy = (K_star_new, K_loc_new(K_star_new)-0.2), size = 20)
  plt.plot(K_saddle_left, q_saddle_left, '--', label = 'New saddle path', color = 'blue')
  plt.plot(K_saddle_right, q_saddle_right, '--', color = 'blue')
  plt.xlabel("K")                       # Add x-axis label
  plt.ylabel("q")                       # Add y-axis label
  plt.title("The dynamics of the permanent increase")  # Add plot title
  plt.grid()                            # Add a grid
  plt.ylim(bottom = 0.5, top = 5)
  plt.xlim(left = 0.5, right = 7)
  # Compute the evolution of K and q
  K = K_trans
  q = q_trans
  # Plot a dashed orange line (notice how it is initialized with empty arrays)
  evolution_line,  = plt.plot([], [], '--', label = 'Transition path', color = 'orange')
  # Define how the evolution_line (the orange dashed line) evolves as a function of time
  def animate(t):
    # Instead of taking steps with linearly increasing size, we use a bit of a
    # trick to take smaller steps in the start and larger steps as time goes by
    step = int((t/50)**3 * len(K))
    # Update the evolution_line data
    evolution_line.set_data(K[:step], q[:step])
    return evolution_line,
  # Add legend and show the plot
  plt.legend()
  return animation.FuncAnimation(fig, animate, frames = 50, interval = 20, blit = True)

In [ ]:
# Obtain the saddle path of the new steady state using the updated function reverse_trajectory_new
# Points to the right of the steady state
K_saddle_right, q_saddle_right = reverse_trajectory_new(K_star_new, T = 80, eps = 0.0001)
# Points to the left of the steady state
K_saddle_left, q_saddle_left = reverse_trajectory_new(K_star_new, T = 79, eps = -0.0001)

# Obtain the the q value and its index on the saddle path
i_interp, q_interp = lin_intpl_q(K_saddle_left, q_saddle_left, K_star_old)

# Set up the transition values of q and K
K_trans = np.concatenate((K_star_old, K_star_old, K_saddle_left[i_interp+1:len(K_saddle_left)].flatten()))
q_trans = np.concatenate((q_star_old, q_interp, q_saddle_left[i_interp+1:len(q_saddle_left)].flatten()))

# Build the animated plot
anim_trans = animated_plot_new(K_trans, q_trans)
plt.close()
anim_trans